# Load Packages

In [1]:
import numpy as np
import numba

#!pip install memory_profiler
import memory_profiler
%load_ext memory_profiler

# Load Data

In [2]:
with np.load('data/demo1.npz') as data:
    x = data['px'][:, 0]

# Find truncation order m
The task is to find the first $m^*$ whereas 

$$
|w_{m^*}| \geq \tau \; \wedge \; |w_{m^*+1}| < \tau
$$

(see "Fixed-Width Window Fracdiff" in Lopez, 2018, pp. 80--84)

In [3]:
# numpy version
def find_truncation_order1(d, tau=1e-5, mmax=20000):
    w = np.empty((mmax+1,))
    w[0] = 1
    for k in range(1, mmax+1):
        w[k] = -w[k-1] * ((d - k + 1) / k)
        if np.abs(w[k]) < tau:
            break
    return k-1, w[:k]

In [4]:
# list version
def find_truncation_order2(d, tau=1e-5, mmax=20000):
    w = [1]
    for k in range(1, mmax+1):
        wk = -w[-1] * ((d - k + 1) / k)
        if abs(wk) < tau:
            break
        w.append(wk)
    return k-1, w

In [5]:
# numba/list version
@numba.jit(nopython=True)
def find_truncation_order3(d, tau=1e-5, mmax=20000):
    w = [1]
    for k in range(1, mmax+1):
        wk = -w[-1] * ((d - k + 1) / k)
        if abs(wk) < tau:
            break
        w.append(wk)
    return k-1, w

# Initial Run
required for numba

In [6]:
d = 0.345
%time mstar, w = find_truncation_order1(d)
%time mstar, w = find_truncation_order2(d)
%time mstar, w = find_truncation_order3(d)

CPU times: user 7.27 ms, sys: 153 µs, total: 7.43 ms
Wall time: 7.51 ms
CPU times: user 1.2 ms, sys: 74 µs, total: 1.27 ms
Wall time: 1.34 ms
CPU times: user 691 ms, sys: 101 ms, total: 792 ms
Wall time: 941 ms


# Speed
The `numba` version is the winner.

In [7]:
d = 0.345
%timeit mstar, w = find_truncation_order1(d)
%timeit mstar, w = find_truncation_order2(d)
%timeit mstar, w = find_truncation_order3(d)

8.91 ms ± 1.27 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)
1.19 ms ± 139 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
117 µs ± 4.44 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


# Memory

In [8]:
d = 0.345
%memit mstar, w = find_truncation_order1(d)
%memit mstar, w = find_truncation_order2(d)
%memit mstar, w = find_truncation_order3(d)

peak memory: 116.12 MiB, increment: -0.23 MiB
peak memory: 115.54 MiB, increment: -0.37 MiB
peak memory: 115.54 MiB, increment: 0.00 MiB


# References
* Prado, M.L. de, 2018. Advances in Financial Machine Learning, 1st ed. Wiley.